<a href="https://colab.research.google.com/github/iamjudy/deep-learning-colab/blob/main/RNN_%E6%83%85%E6%84%8F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入深度學習套件

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

### 2. 讀入數據

一般自然語言處理, 我們會限制最大要使用的字數。

In [ ]:
# 用 10000 字
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
print(len(x_train))
print(len(x_test))

25000
25000


In [ ]:
# 注意每筆評論的長度當然是不一樣的。
print(len(x_train[0]))
print(len(x_train[1]))

218
189


In [ ]:
 # 正評 or 負評
print(y_train[0])
print(y_train[1])

1
0


### 3. 資料處理

雖然我們可以做真的 seq2seq, 可是資料長度不一樣對計算上有麻煩, 因此平常還是會固定一定長度, 其餘補 0。

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

### 4. step 01: 打造一個函數學習機

In [ ]:
model = Sequential()
model.add(Embedding(10000, 256)) # shape 256 維

# 減少 LSTM 數量 128 --> 64
model.add(LSTM(64))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         2560000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                82176     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,642,241
Trainable params: 2,642,241
Non-trainable params: 0
_________________________________________________________________


### 5. step 02: 訓練

In [ ]:
# 減少至 5 epochs
model.fit(x_train, y_train, batch_size=32, epochs=5,
         validation_data=(x_test, y_test))

Epoch 1/5
782/782 [==============================] - 29s 35ms/step - loss: 0.4068 - accuracy: 0.8144 - val_loss: 0.3408 - val_accuracy: 0.8490
Epoch 2/5
782/782 [==============================] - 31s 39ms/step - loss: 0.2510 - accuracy: 0.8981 - val_loss: 0.3449 - val_accuracy: 0.8475
Epoch 3/5
782/782 [==============================] - 31s 39ms/step - loss: 0.1717 - accuracy: 0.9351 - val_loss: 0.4191 - val_accuracy: 0.8481
Epoch 4/5
782/782 [==============================] - 27s 34ms/step - loss: 0.1160 - accuracy: 0.9582 - val_loss: 0.5173 - val_accuracy: 0.8378
Epoch 5/5
782/782 [==============================] - 27s 34ms/step - loss: 0.1009 - accuracy: 0.9634 - val_loss: 0.5067 - val_accuracy: 0.8376


### 6. step 03: 測試

In [ ]:
from tensorflow.keras.datasets.imdb import get_word_index

In [ ]:
word_index = get_word_index()
word_index['this']
text = "this movie is worth seeing" 
seq = [word_index[x] for x in text.split()]
print(model.predict([seq]))

1654784/1641221 [==============================] - 0s 0us/step
[[0.96698916]]


In [ ]:
text = "this genre of movie is boring" # 電影種類很無聊 -> 差評！
seq = [word_index[x] for x in text.split()]
print(model.predict([seq]))

[[0.23855028]]
